## Imports

In [ ]:
import pmp_functions_v4 as pmp
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

path = "../../Data_Ryan"

## Global Variables

In [389]:
frequency = 1
t_cost = 0
short = True
beta_neutral = False
target_vol = 0.10
target_vol_monthly = target_vol / np.sqrt(12)
min_regions = 4
k = 2

## Data

### Riskfree Data

In [390]:
# --- Load Riskfree Rate ---
factors_data = pd.read_excel(
    f"{path}/Factors.xlsx",
    index_col = 0,
    parse_dates = True
)

factors_data.index = pd.to_datetime(factors_data.index, format='%Y%m')
factors_data.index = factors_data.index + pd.offsets.MonthEnd(0)
factors_data /= 100

riskfree = factors_data["RF"].resample('ME').last()
riskfree

/var/folders/37/zcyd0byn17lfrmxtt4_4k3s40000gn/T/ipykernel_63159/2677495952.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  factors_data = pd.read_excel(


1926-07-31    0.0022
1926-08-31    0.0025
1926-09-30    0.0023
1926-10-31    0.0032
1926-11-30    0.0031
               ...  
2025-06-30    0.0034
2025-07-31    0.0034
2025-08-31    0.0038
2025-09-30    0.0033
2025-10-31    0.0037
Freq: ME, Name: RF, Length: 1192, dtype: float64

### Factor Data

In [391]:
# --- Load Factors Data ---
famafrench_data = pd.read_csv(
    f"{path}/famafrench_factors.csv",
    index_col = 0,
    parse_dates = True
)

famafrench_data.index = pd.to_datetime(famafrench_data.index, format='%Y%m')
famafrench_data.index = famafrench_data.index + pd.offsets.MonthEnd(0)
famafrench_data.dropna(inplace=True)
famafrench_data

/var/folders/37/zcyd0byn17lfrmxtt4_4k3s40000gn/T/ipykernel_63159/297690613.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  famafrench_data = pd.read_csv(


,MKT-RF,SMB,HML,RMW,CMA,UMD,BAB
DATE,,,,,,,
1980-01-31,0.0550,0.0188,0.0185,-0.0184,0.0189,0.0745,0.0695
1980-02-29,-0.0123,-0.0162,0.0059,-0.0095,0.0292,0.0789,-0.0132
1980-03-31,-0.1289,-0.0697,-0.0096,0.0182,-0.0105,-0.0958,-0.1181
1980-04-30,0.0396,0.0105,0.0103,-0.0218,0.0034,-0.0048,0.0574
1980-05-31,0.0526,0.0200,0.0038,0.0043,-0.0063,-0.0118,0.0618
...,...,...,...,...,...,...,...
2025-05-31,0.0606,-0.0072,-0.0288,0.0129,0.0251,0.0221,0.0256
2025-06-30,0.0486,-0.0002,-0.0160,-0.0320,0.0145,-0.0264,0.0527
2025-07-31,0.0198,-0.0015,-0.0127,-0.0029,-0.0208,-0.0096,0.0184


### Benchmark Data

In [392]:
# --- Benchmark Data ---
benchmark_data = pd.read_excel(
    f"{path}/Benchmarks.xlsx",
    index_col = 0,
    parse_dates = True
)

benchmark_data.index = pd.to_datetime(benchmark_data.index)
benchmark_data = benchmark_data.resample('ME').last()


benchmark_return = benchmark_data[['MSCI World']].pct_change()
benchmark_return = benchmark_return.squeeze()
benchmark_return

Date
1986-12-31         NaN
1987-01-31         NaN
1987-02-28         NaN
1987-03-31         NaN
1987-04-30         NaN
                ...   
2025-07-31    0.013121
2025-08-31    0.026408
2025-09-30    0.032574
2025-10-31    0.020226
2025-11-30    0.003149
Freq: ME, Name: MSCI World, Length: 468, dtype: float64

### Macro Data

In [393]:
# --- Load Macro Data ---
CPI_forecasts = pd.read_excel(
    f"{path}/Inflation_forecasts.xlsx",
    index_col = 0,
    parse_dates = True
)
CPI_forecasts.index = pd.to_datetime(CPI_forecasts.index)
CPI_forecasts.index = CPI_forecasts.index + pd.offsets.MonthEnd(0)
CPI_forecasts *= 100

CPI_forecasts

,UK,CH,JP,AU,EU,US
Date,,,,,,
1970-01-31,4.95739,2.31338,7.75127,NaN,NaN,NaN
1970-02-28,4.93065,2.12360,7.75127,NaN,NaN,NaN
1970-03-31,5.14198,2.49918,7.71710,NaN,NaN,NaN
1970-04-30,5.61884,2.59281,7.68362,NaN,NaN,NaN
1970-05-31,6.08365,3.13683,7.24558,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,4.10000,0.10000,3.30000,2.1,1.96760,3.17
2025-07-31,4.20000,0.20000,3.10000,3.2,2.00791,2.79
2025-08-31,4.10000,0.20000,2.70000,3.2,2.02889,2.79


In [394]:
RGDP_forecasts = pd.read_excel(
    f"{path}/RGDP_forecasts.xlsx",
    index_col = 0,
    parse_dates = True
)
RGDP_forecasts.index = pd.to_datetime(RGDP_forecasts.index)
RGDP_forecasts.index = RGDP_forecasts.index + pd.offsets.MonthEnd(0)
RGDP_forecasts *= 100

RGDP_forecasts

,AU,UK,CH,JP,EU,US
Date,,,,,,
1970-07-31,3.15789,NaN,NaN,NaN,NaN,2.936175
1970-08-31,3.15789,NaN,NaN,NaN,NaN,2.936175
1970-09-30,3.15789,NaN,NaN,NaN,NaN,2.936175
1970-10-31,5.26317,NaN,NaN,NaN,NaN,2.994250
1970-11-30,5.26317,NaN,NaN,NaN,NaN,2.994250
...,...,...,...,...,...,...
2025-06-30,2.10000,2.5,2.439516,1.976375,1.522007,1.345350
2025-07-31,3.20000,2.5,1.284580,1.074700,1.350512,1.450175
2025-08-31,3.20000,2.5,1.284580,1.074700,1.350512,1.450175


### Bond Data

In [395]:
# --- Load Bond Futures ---
bond_futures = pd.read_excel(
    f"{path}/Bond Futures.xlsx",
    index_col = 0,
    parse_dates = True
)
bond_futures.index = pd.to_datetime(bond_futures.index)
bond_futures.index = bond_futures.index + pd.offsets.MonthEnd(0)

bond_futures

,EU,JP,AU,US,CH,EM,UK
Dates,,,,,,,
1998-01-31,72.58774,89.88,83.91110,58.968750,90.12,NaN,89.28422
1998-02-28,73.84924,90.45,83.86650,58.437500,91.37,NaN,89.20823
1998-03-31,72.76451,90.75,84.08059,58.281250,90.54,NaN,92.70965
1998-04-30,74.66254,91.56,83.99135,58.250000,89.35,NaN,92.89550
1998-05-31,75.80680,93.14,84.46878,58.593750,91.27,NaN,91.67311
...,...,...,...,...,...,...,...
2025-07-31,147.00613,137.03,61.55103,111.078125,196.99,15.03735,121.79588
2025-08-31,150.42250,136.54,62.63541,112.468750,200.93,15.12679,122.26090
2025-09-30,151.15975,135.79,63.31735,112.484375,205.32,15.14941,122.26156


In [396]:
bond_returns = bond_futures.pct_change()
bond_returns

,EU,JP,AU,US,CH,EM,UK
Dates,,,,,,,
1998-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-28,0.017379,0.006342,-0.000532,-0.009009,0.013870,NaN,-0.000851
1998-03-31,-0.014688,0.003317,0.002553,-0.002674,-0.009084,NaN,0.039250
1998-04-30,0.026085,0.008926,-0.001061,-0.000536,-0.013143,NaN,0.002005
1998-05-31,0.015326,0.017256,0.005684,0.005901,0.021489,NaN,-0.013159
...,...,...,...,...,...,...,...
2025-07-31,-0.032262,-0.007101,-0.022422,-0.009475,-0.015542,-0.009048,-0.044820
2025-08-31,0.023240,-0.003576,0.017618,0.012519,0.020001,0.005948,0.003818
2025-09-30,0.004901,-0.005493,0.010887,0.000139,0.021848,0.001495,0.000005


## Signal Generation

In [397]:
# --- Compute Business Cycle Signal ---
CPI_component = CPI_forecasts.diff(12)
RGDP_component = RGDP_forecasts.diff(12)
business_cyle_signal = - (0.5 * RGDP_component) - (0.5 * CPI_component)
business_cyle_signal = business_cyle_signal.resample('ME').last()
business_cyle_signal = business_cyle_signal.dropna(how='all')
business_cyle_signal.tail()

,AU,CH,EU,JP,UK,US
Date,,,,,,
2025-06-30,1.71445,-0.564448,-0.224851,-1.728625,-1.008965,0.069925
2025-07-31,-0.39146,0.867678,0.092139,-0.488158,-1.438370,-0.026338
2025-08-31,-0.39146,0.752568,-0.131066,-0.188158,-1.354405,-0.026338
2025-09-30,-0.39146,0.643893,-0.431266,-0.538158,-1.593315,-0.026338
2025-10-31,-1.37530,0.606678,-0.055999,-0.325256,-1.275500,-0.061275


In [398]:
def make_country_weights_ls_vol(
    signal,
    returns,
    k=1,
    signal_lag=0,
    min_regions=1,
    vol_target=0.10,        # Target volatility (10% p.a.)
    vol_lookback=36,        # 36 months
):
    """
    FINAL VERSION:
    ---------------------------
    - Per-region history check: region must have >= vol_lookback valid monthly returns.
    - Cross-sectional Z-score standardization (only on eligible regions).
    - Long top-k, short bottom-k.
    - Dollar-neutral raw weights.
    - No gross-exposure normalization (vol targeting determines leverage).
    - Scaling starts ONLY once we have >= vol_lookback strategy returns.
    - Remove ALL invalid dates completely (no zero-filling).
    """

    import pandas as pd
    import numpy as np

    # -----------------------------
    # 1. Align data & optional lag
    # -----------------------------
    if signal_lag > 0:
        signal = signal.shift(signal_lag)

    ranks = signal.rank(axis=1, ascending=False)

    idx = ranks.index.union(returns.index)
    ranks = ranks.reindex(idx)
    returns = returns.reindex(idx)

    regions = ranks.columns

    # -----------------------------
    # 2. Eligibility per region
    # -----------------------------
    # (a) region must have >= 36 real returns
    enough_history = (
        returns.notna()
        .rolling(vol_lookback)
        .sum()
        >= vol_lookback
    )

    # (b) need next-month return
    has_next = returns.shift(-1).notna()

    eligible = enough_history & has_next

    # require at least min_regions eligible
    valid_dates = eligible.sum(axis=1) >= min_regions

    # -----------------------------
    # 3. Build raw long/short weights
    # -----------------------------
    out = []

    for t in eligible.index[valid_dates]:

        good = eligible.loc[t]
        usable_cols = good[good].index

        # ranks for eligible regions only
        r_t = ranks.loc[t, usable_cols].dropna()
        n = len(r_t)
        if n < min_regions:
            continue

        # ---- Z-score standardization ----
        mean = r_t.mean()
        std = r_t.std() if r_t.std() != 0 else 1
        z = (r_t - mean) / std

        # top-k long / bottom-k short
        k_eff = min(k, n // 2)
        winners = z.nsmallest(k_eff).index
        losers  = z.nlargest(k_eff).index

        # initialize EMPTY weight vector
        w = pd.Series(0.0, index=regions)

        # assign long/short only on eligible regions
        w[winners] = +1
        w[losers]  = -1

        # Dollar-neutral
        w = w - w.mean()

        out.append(w.rename(t))

    # create DataFrame with *only valid rows*
    weights_raw = pd.DataFrame(out)

    # -----------------------------
    # 4. Volatility Targeting
    # -----------------------------
    # unscaled strategy return
    returns = returns.reindex(weights_raw.index)
    strat_ret_raw = (weights_raw.shift(1) * returns).sum(axis=1)

    # realized vol (annualized)
    realized_vol = strat_ret_raw.rolling(vol_lookback).std() * np.sqrt(12)

    # scaling factor, but only valid after vol_lookback returns
    scaling = vol_target / realized_vol
    scaling = scaling.replace([np.inf, -np.inf], np.nan)

    # scaling starts only after enough history
    scaling = scaling.where(realized_vol.notna(), 1)

    weights_scaled = weights_raw.mul(scaling, axis=0)

    return weights_scaled, scaling, weights_raw


In [399]:
def make_country_weights_ls_vol(
    signal,
    returns,
    signal_lag=0,
    min_regions=1,
    vol_target=0.10,        # 10% annualized target vol
    vol_lookback=36,        # 3-year rolling window
):
    """
    FINAL CROSS-SECTIONAL Z-SCORE COUNTRY STRATEGY
    ----------------------------------------------
    - Region eligible only if it has >= vol_lookback valid past returns
      AND next-month return available.
    - Only eligible regions are used for ranking and standardization.
    - Weights = standardized Z-scores (NO Dollar-neutral step needed).
    - No gross-exposure normalization; vol-targeting determines exposure.
    - Scaling only active once strategy has >= vol_lookback monthly returns.
    - Only valid dates remain in output (no zero-filling).
    """

    import pandas as pd
    import numpy as np

    # -----------------------------
    # 1. Align data & optional lag
    # -----------------------------
    if signal_lag > 0:
        signal = signal.shift(signal_lag)

    ranks = signal.rank(axis=1, ascending=False)

    idx = ranks.index.union(returns.index)
    ranks = ranks.reindex(idx)
    returns = returns.reindex(idx)

    regions = ranks.columns

    # -----------------------------
    # 2. Eligibility per region (strict)
    # -----------------------------
    # (a) Region must have >=36 valid past returns
    enough_history = (
        returns.notna()
        .rolling(vol_lookback)
        .sum()
        >= vol_lookback
    )

    # (b) Need next-month return
    has_next = returns.shift(-1).notna()

    eligible = enough_history & has_next

    # global condition: at least min_regions usable
    valid_dates = eligible.sum(axis=1) >= min_regions

    # -----------------------------
    # 3. Build RAW Z-score weights
    # -----------------------------
    out = []

    for t in eligible.index[valid_dates]:

        # eligible regions at time t
        good = eligible.loc[t]
        usable_cols = good[good].index

        # restrict ranks to eligible
        r_t = ranks.loc[t, usable_cols].dropna()
        n = len(r_t)
        if n < min_regions:
            continue

        # --- Cross-sectional Z-score ---
        mean = r_t.mean()
        std = r_t.std() if r_t.std() != 0 else 1
        z = (r_t - mean) / std

        # embed into full index
        w = pd.Series(np.nan, index=regions)
        w.loc[usable_cols] = z

        out.append(w.rename(t))

    # DataFrame without invalid dates
    weights_raw = pd.DataFrame(out)

    # -----------------------------
    # 4. Volatility Targeting
    # -----------------------------
    # align returns
    returns_aligned = returns.reindex(weights_raw.index)

    # unscaled strategy returns
    strat_ret_raw = (weights_raw.shift(1) * returns_aligned).sum(axis=1)

    # rolling realized vol (annualized)
    realized_vol = strat_ret_raw.rolling(vol_lookback).std() * np.sqrt(12)

    # scaling factor
    scaling = vol_target / realized_vol
    scaling = scaling.replace([np.inf, -np.inf], np.nan)

    # scaling starts only after enough history
    scaling = scaling.where(realized_vol.notna(), 1.0)

    weights_scaled = weights_raw.mul(scaling, axis=0)

    return weights_scaled, scaling, weights_raw


In [400]:
bond_returns = bond_returns[["EU", "JP", "AU", "US", "CH", "UK"]]
bond_returns.head()

,EU,JP,AU,US,CH,UK
Dates,,,,,,
1998-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-28,0.017379,0.006342,-0.000532,-0.009009,0.013870,-0.000851
1998-03-31,-0.014688,0.003317,0.002553,-0.002674,-0.009084,0.039250
1998-04-30,0.026085,0.008926,-0.001061,-0.000536,-0.013143,0.002005
1998-05-31,0.015326,0.017256,0.005684,0.005901,0.021489,-0.013159


In [401]:
weights, scaling_factors, weights_raw = make_country_weights_ls_vol(
    signal=business_cyle_signal,
    returns=bond_returns,
    min_regions=min_regions,
    signal_lag=0,
    vol_target=target_vol,
    vol_lookback=36
)

In [402]:
weights_raw

,AU,CH,EU,JP,UK,US
2001-01-31,1.336306,-1.336306,-0.801784,0.267261,0.801784,-0.267261
2001-02-28,1.336306,-1.336306,-0.801784,0.267261,0.801784,-0.267261
2001-03-31,1.336306,-1.336306,-0.801784,-0.267261,0.801784,0.267261
2001-04-30,1.336306,-1.336306,-0.267261,-0.801784,0.801784,0.267261
2001-05-31,1.336306,-1.336306,0.267261,-0.801784,0.801784,-0.267261
...,...,...,...,...,...,...
2025-06-30,-1.336306,0.267261,-0.267261,1.336306,0.801784,-0.801784
2025-07-31,0.267261,-1.336306,-0.801784,0.801784,1.336306,-0.267261
2025-08-31,0.801784,-1.336306,-0.267261,0.267261,1.336306,-0.801784
2025-09-30,-0.267261,-1.336306,0.267261,0.801784,1.336306,-0.801784


In [403]:
weights

,AU,CH,EU,JP,UK,US
2001-01-31,1.336306,-1.336306,-0.801784,0.267261,0.801784,-0.267261
2001-02-28,1.336306,-1.336306,-0.801784,0.267261,0.801784,-0.267261
2001-03-31,1.336306,-1.336306,-0.801784,-0.267261,0.801784,0.267261
2001-04-30,1.336306,-1.336306,-0.267261,-0.801784,0.801784,0.267261
2001-05-31,1.336306,-1.336306,0.267261,-0.801784,0.801784,-0.267261
...,...,...,...,...,...,...
2025-06-30,-0.823155,0.164631,-0.164631,0.823155,0.493893,-0.493893
2025-07-31,0.166407,-0.832037,-0.499222,0.499222,0.832037,-0.166407
2025-08-31,0.501019,-0.835031,-0.167006,0.167006,0.835031,-0.501019
2025-09-30,-0.191549,-0.957746,0.191549,0.574647,0.957746,-0.574647


In [404]:
results = pmp.run_cc_strategy(
    weights = weights,
    returns = bond_returns,
    rf = riskfree,
    frequency = frequency,
    t_cost = t_cost, 
    benchmark = benchmark_return,
    long_short = short,
    beta_neutral = beta_neutral
)

display(results)

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,
2001-02-28,-0.005467,-0.005467,-0.084288,0.000000,0.0,0.0038,0.557383,-0.556156,-0.331751,0.112840,0.329777,-0.112093
2001-03-31,-0.000682,-0.000682,-0.065527,1.405351,0.0,0.0042,0.558363,-0.566716,-0.321305,0.112330,0.329307,-0.111979
2001-04-30,0.010802,0.010802,0.074300,1.629660,0.0,0.0039,0.559460,-0.553333,-0.332993,-0.113674,0.331174,0.109366
2001-05-31,-0.009614,-0.009614,-0.012221,1.471083,0.0,0.0032,0.556961,-0.558056,-0.104919,-0.337026,0.330947,0.112092
2001-06-30,-0.026084,-0.026084,-0.030863,1.622362,0.0,0.0028,0.555882,-0.559320,0.113247,-0.331412,0.330871,-0.109267
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,-0.006207,-0.006207,0.043488,0.478295,0.0,0.0034,-0.557765,0.112298,-0.111565,0.547688,0.340014,-0.330670
2025-07-31,-0.002093,-0.002093,0.013121,0.481679,0.0,0.0034,-0.553889,0.111846,-0.109470,0.563054,0.325100,-0.336641
2025-08-31,-0.026003,-0.026003,0.026408,1.397467,0.0,0.0038,0.114636,-0.557326,-0.334319,0.331204,0.554160,-0.108355


In [405]:
pmp.run_perf_summary_benchmark_vs_strategy(results, alreadyXs = True)

,Benchmark,Strategy
Arithm Avg Total Return,8.7270,0.8561
Arithm Avg Xs Return,7.0543,-0.8166
Std Xs Returns,15.4607,11.0649
Sharpe Arithmetic,0.4563,-0.0738
Geom Avg Total Return,7.7844,0.2311
Geom Avg Xs Return,6.1002,-1.4531
Sharpe Geometric,0.3946,-0.1313
Min Xs Return,-19.0140,-17.1962
Max Xs Return,12.8184,8.5400
Skewness,-0.6097,-0.7522
